In [1]:
# import packages
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# colormap stuff for plot generation
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


# Isochrone parameters
The difference this time is that we are using 6 filters for chi-squared analysis. The interpolation code will most definitely need to be changed so that this can be performed on the synethic CMD outputs created from Nbody6

In [2]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F150W', 'jwst,F200W', 'jwst,F277W', 'jwst,F356W', 'jwst,F444W']
filters = ['m_jwst_F115W', 'm_jwst_F150W', 'm_jwst_F200W', 'm_jwst_F277W', 'm_jwst_F356W', 'm_jwst_F444W']
iso_dir = 'isochrones/'
metallicity = 0

In [3]:
# define log age array for isochrone grid
level_ages = np.linspace(1, 10, 19)
level_ages = np.multiply(level_ages, 1e6)
print(level_ages)
log_age_arr = np.log10(level_ages)
print(log_age_arr)

[ 1000000.  1500000.  2000000.  2500000.  3000000.  3500000.  4000000.
  4500000.  5000000.  5500000.  6000000.  6500000.  7000000.  7500000.
  8000000.  8500000.  9000000.  9500000. 10000000.]
[6.         6.17609126 6.30103    6.39794001 6.47712125 6.54406804
 6.60205999 6.65321251 6.69897    6.74036269 6.77815125 6.81291336
 6.84509804 6.87506126 6.90308999 6.92941893 6.95424251 6.97772361
 7.        ]


# Generate isochrone grid

In [4]:
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

isochrone generation done


# Function for calculating chi-square value based on photometric magnitudes

In [5]:
print(instances[0].points.keys())

['L', 'Teff', 'R', 'mass', 'logg', 'isWR', 'mass_current', 'phase', 'm_jwst_F115W', 'm_jwst_F150W', 'm_jwst_F200W', 'm_jwst_F277W', 'm_jwst_F356W', 'm_jwst_F444W']


In [6]:
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    
    for i in range(len(iso_grid)):
        for j in range(len(iso_grid[i].points)):
            arr = []
            chi_square = 0
            for k in range(len(sample_mags)):
                mag_obs = sample_mags[k]
                mag_exp = iso_grid[i].points[j][filters[k]]
                chi_square = chi_square + (((mag_obs - mag_exp) ** 2) / mag_exp)

            arr.append(chi_square)
            arr.append(iso_grid[i].points[j]['mass'])
            arr.append(10 ** log_age_arr[i])
            
            results.append(arr)
    
    sorted_results = sorted(results, key=sort_by_chi_square)
    
    report(sorted_results[:10])
    
def sort_by_chi_square(sub_list):
    return sub_list[0]

def report(output_arr):
    for i in range(len(output_arr)):
        print("Chi-square: " + str(output_arr[i][0]))
        print("Mass (Msun): " + str(output_arr[i][1]))
        print("Age (Myr): " + str(output_arr[i][2]))
        print()

# Test chi-square function against a pre-determined star
I am going to perform this test on a 0.9 Msun, 1 Myr star - we expect a 1 Msun, 1 Myr star to have a very low chi-square value

In [7]:
idx = np.where( abs(instances[0].points['mass'] - 0.90) == min(abs(instances[0].points['mass'] - 0.90)) )[0]

sample_mags = []

for i in range(len(filters)):
    sample_mags.append(instances[0].points[idx][filters[i]][0])
    
print(sample_mags)

[10.984540911059824, 10.294770171751042, 9.977909538052804, 9.972149774893902, 9.907517984661238, 9.943493220686026]


In [8]:
chi_square_reverse_model(instances, sample_mags)

Chi-square: 0.0
Mass (Msun): 0.9
Age (Myr): 1000000.0

Chi-square: 0.0010994548742689148
Mass (Msun): 1.1
Age (Myr): 1500000.0000000005

Chi-square: 0.0020831283148331475
Mass (Msun): 1.3
Age (Myr): 2000000.0000000002

Chi-square: 0.004739632134347281
Mass (Msun): 1.2
Age (Myr): 1500000.0000000005

Chi-square: 0.004774167795039644
Mass (Msun): 1.4
Age (Myr): 2499999.9999999995

Chi-square: 0.005655083306777714
Mass (Msun): 1.4
Age (Myr): 2000000.0000000002

Chi-square: 0.011334603727357336
Mass (Msun): 1.0
Age (Myr): 1000000.0

Chi-square: 0.0122105561068105
Mass (Msun): 1.2
Age (Myr): 2000000.0000000002

Chi-square: 0.013012026695863904
Mass (Msun): 0.8
Age (Myr): 1000000.0

Chi-square: 0.01397495683593276
Mass (Msun): 1.0
Age (Myr): 1500000.0000000005

